In [77]:
import pandas as pd
import numpy as np
import feather
from sklearn.preprocessing import Imputer

In [78]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

## Ames - Iowa House Prediction Kaggle Competition
This is my first draft of the [kaggle house prediction competition](https://www.kaggle.com/c/house-prices-advanced-regression-techniques). 

In [79]:
df = pd.read_csv('./train.csv')
df_test = pd.read_csv('./test.csv')

### Preprocessing the data
There are a lot of variables in the dataset, and I noticed two main types of categorical variables, ones which have order or rank (`OverallQual`, `ExterCond`), these are variables relating to quality and condition. The other types are unordered labels, like what type of heating the house has, or the material used. 

Since we need to change this data into numerical format, it needs to be done in such a way that the ordered variables have a numerical rank system mapped to numbers (1 = Poor, and 5 = Excellent). The unordered can just be converted into categorical codes. 

There are also labels that could be [one hot encoded](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f) as they don't have too many variables next to their name and can be seperated.
And for this draft I also decided to remove the `Neighborhood` variable as there were too many unique neighborhoods in this dataset and could cause too much noise, a later task would be to so some research on the different neighborhoods and assign each one a weight corresponding to the wealth of the area."

I Also deided to remove `Alley` because this column was filled with NaNs and instead of trying to impute it later down the line, I made the assumption that it won't have a big impact on the predictions anyway.

In [80]:
def preprocess_data(df):
    
    if('Neighborhood' in df.columns):
        df.drop('Neighborhood', axis=1, inplace=True)
    
    if('Alley' in df.columns):
        df.drop('Alley', axis=1, inplace=True)
        
    unordered_labels = ['MSZoning',
                   'Condition1','Condition2','HouseStyle','RoofMatl','Exterior1st',
                    'Exterior2nd','Heating', 'Electrical','Functional',
                    'GarageType','MiscFeature','SaleType','SaleCondition']

    ordered_labels = ['ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure',
                      'BsmtFinType1','BsmtFinType2','HeatingQC',
                     'KitchenQual','FireplaceQu','GarageQual','GarageCond','PoolQC','Fence']

    ohe_labels = ['CentralAir','GarageFinish','PavedDrive','Street', 'LotShape','LandContour','Utilities',
                  'LotConfig','LandSlope','BldgType','RoofStyle','MasVnrType',
                  'Foundation']

    pool_quality_order = ['Fa','TA','Gd','Ex']
    quality_order = ['Po','Fa','TA','Gd','Ex']
    fence_order = ['MnWw','GdWo','MnPrv','GdPrv']
    basement_exp = ['No','Mn','Av','Gd']
    fin_types = ['Unf','LwQ','Rec','BLQ','ALQ','GLQ']
    
    for c in unordered_labels:
        if(c in df.columns):
            df[c] = df[c].astype('category').cat.codes
    
    for c in ordered_labels:
        
        if(c in df.columns):
            
            if(c == 'PoolQC'):
                df[c] = df[c].astype('category', ordered=True, categories=pool_quality_order).cat.codes
            elif(c == 'BsmtFinType1' or c == 'BsmtFinType2'):
                df[c] = df[c].astype('category', ordered=True, categories=fin_types).cat.codes
            elif(c == 'BsmtExposure'):
                df[c] = df[c].astype('category', ordered=True, categories=basement_exp).cat.codes
            elif(c == 'Fence'):
                df[c] = df[c].astype('category', ordered=True, categories=fence_order).cat.codes
            else:
                df[c] = df[c].astype('category', ordered=True, categories=quality_order).cat.codes
            
            
    df = pd.get_dummies(data=df, columns=ohe_labels)
                
    return df


#### IGNORE THE WEIRD WARNING ... JUST PANDAS UPDATES

In [81]:
df = preprocess_data(df)

/Users/joshishushruth/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:45: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
/Users/joshishushruth/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:41: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
/Users/joshishushruth/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:39: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
/Users/joshishushruth/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:37: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
/Users/joshishushruth/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:43: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead


In [82]:
display_all(df.head())

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Condition1,Condition2,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofMatl,Exterior1st,Exterior2nd,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageCars,GarageArea,GarageQual,GarageCond,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,CentralAir_N,CentralAir_Y,GarageFinish_Fin,GarageFinish_RFn,GarageFinish_Unf,PavedDrive_N,PavedDrive_P,PavedDrive_Y,Street_Grvl,Street_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_AllPub,Utilities_NoSeWa,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood
0,1,60,3,65.0,8450,2,2,5,7,5,2003,2003,1,12,13,196.0,3,2,3,2,0,5,706,0,0,150,856,1,4,4,856,854,0,1710,1,0,2,1,3,1,3,8,6,0,-1,1,2003.0,2,548,2,2,0,61,0,0,0,0,-1,-1,-1,0,2,2008,8,4,208500,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0
1,2,20,3,80.0,9600,1,2,2,6,8,1976,1976,1,8,8,0.0,2,2,3,2,3,4,978,0,0,284,1262,1,4,4,1262,0,0,1262,0,1,2,0,3,1,2,6,6,1,2,1,1976.0,2,460,2,2,298,0,0,0,0,0,-1,-1,-1,0,5,2007,8,4,181500,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0
2,3,60,3,68.0,11250,2,2,5,7,5,2001,2002,1,12,13,162.0,3,2,3,2,1,5,486,0,0,434,920,1,4,4,920,866,0,1786,1,0,2,1,3,1,3,6,6,1,2,1,2001.0,2,608,2,2,0,42,0,0,0,0,-1,-1,-1,0,9,2008,8,4,223500,0,1,0,1,0,0,0,1,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0
3,4,70,3,60.0,9550,2,2,5,7,5,1915,1970,1,13,15,0.0,2,2,2,3,0,4,216,0,0,540,756,1,3,4,961,756,0,1717,1,0,1,0,3,1,3,7,6,1,3,5,1998.0,3,642,2,2,0,35,272,0,0,0,-1,-1,-1,0,2,2006,8,0,140000,0,1,0,0,1,0,0,1,0,1,1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0
4,5,60,3,84.0,14260,2,2,5,8,5,2000,2000,1,12,13,350.0,3,2,3,2,2,5,655,0,0,490,1145,1,4,4,1145,1053,0,2198,1,0,2,1,4,1,3,9,6,1,2,1,2000.0,3,836,2,2,192,84,0,0,0,0,-1,-1,-1,0,12,2008,8,4,250000,0,1,0,1,0,0,0,1,0,1,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0


In [83]:
df.isnull().sum()

Id                      0
MSSubClass              0
MSZoning                0
LotFrontage           259
LotArea                 0
Condition1              0
Condition2              0
HouseStyle              0
OverallQual             0
OverallCond             0
YearBuilt               0
YearRemodAdd            0
RoofMatl                0
Exterior1st             0
Exterior2nd             0
MasVnrArea              8
ExterQual               0
ExterCond               0
BsmtQual                0
BsmtCond                0
BsmtExposure            0
BsmtFinType1            0
BsmtFinSF1              0
BsmtFinType2            0
BsmtFinSF2              0
BsmtUnfSF               0
TotalBsmtSF             0
Heating                 0
HeatingQC               0
Electrical              0
                     ... 
Utilities_NoSeWa        0
LotConfig_Corner        0
LotConfig_CulDSac       0
LotConfig_FR2           0
LotConfig_FR3           0
LotConfig_Inside        0
LandSlope_Gtl           0
LandSlope_Mo

### Imputation
Now that we have a numerical representation of all our variables, there are still a few of NaN's we need to account for. We could just remove the NaN's altogether, but since the dataset it quite related to each other (ie all houses are in the same region-ish) we could just calculate the mean and assign it that.

In [73]:
def impute_nans(df):
    fill_NaN = Imputer(missing_values=np.nan, strategy='mean', axis=1)
    imputed_DF = pd.DataFrame(fill_NaN.fit_transform(df))
    imputed_DF.columns = df.columns
    imputed_DF.index = df.Id
    
    return imputed_DF

In [74]:
df = impute_nans(df)

In [75]:
df['Id'] = df['Id'].astype(int)

year_cols = ['YearBuilt','YearRemodAdd','GarageYrBlt','YrSold']
df[year_cols] = df[year_cols].astype(int)

display_all(df.head())

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Condition1,Condition2,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofMatl,Exterior1st,Exterior2nd,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageCars,GarageArea,GarageQual,GarageCond,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,CentralAir_N,CentralAir_Y,GarageFinish_Fin,GarageFinish_RFn,GarageFinish_Unf,PavedDrive_N,PavedDrive_P,PavedDrive_Y,Street_Grvl,Street_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_AllPub,Utilities_NoSeWa,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,60.0,3.0,65.0,8450.0,2.0,2.0,5.0,7.0,5.0,2003,2003,1.0,12.0,13.0,196.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,706.0,-1.0,0.0,150.0,856.0,1.0,-1.0,5.0,856.0,854.0,0.0,1710.0,1.0,0.0,2.0,1.0,3.0,1.0,-1.0,8.0,6.0,0.0,-1.0,2.0,2003,2.0,548.0,-1.0,-1.0,0.0,61.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,2.0,2008,8.0,4.0,208500.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2,20.0,3.0,80.0,9600.0,1.0,2.0,2.0,6.0,8.0,1976,1976,1.0,8.0,8.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,978.0,-1.0,0.0,284.0,1262.0,1.0,-1.0,5.0,1262.0,0.0,0.0,1262.0,0.0,1.0,2.0,0.0,3.0,1.0,-1.0,6.0,6.0,1.0,-1.0,2.0,1976,2.0,460.0,-1.0,-1.0,298.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,5.0,2007,8.0,4.0,181500.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,3,60.0,3.0,68.0,11250.0,2.0,2.0,5.0,7.0,5.0,2001,2002,1.0,12.0,13.0,162.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,486.0,-1.0,0.0,434.0,920.0,1.0,-1.0,5.0,920.0,866.0,0.0,1786.0,1.0,0.0,2.0,1.0,3.0,1.0,-1.0,6.0,6.0,1.0,-1.0,2.0,2001,2.0,608.0,-1.0,-1.0,0.0,42.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,9.0,2008,8.0,4.0,223500.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,4,70.0,3.0,60.0,9550.0,2.0,2.0,5.0,7.0,5.0,1915,1970,1.0,13.0,15.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,216.0,-1.0,0.0,540.0,756.0,1.0,-1.0,5.0,961.0,756.0,0.0,1717.0,1.0,0.0,1.0,0.0,3.0,1.0,-1.0,7.0,6.0,1.0,-1.0,6.0,1998,3.0,642.0,-1.0,-1.0,0.0,35.0,272.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,2.0,2006,8.0,0.0,140000.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5,5,60.0,3.0,84.0,14260.0,2.0,2.0,5.0,8.0,5.0,2000,2000,1.0,12.0,13.0,350.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,655.0,-1.0,0.0,490.0,1145.0,1.0,-1.0,5.0,1145.0,1053.0,0.0,2198.0,1.0,0.0,2.0,1.0,4.0,1.0,-1.0,9.0,6.0,1.0,-1.0,2.0,2000,3.0,836.0,-1.0,-1.0,192.0,84.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,12.0,2008,8.0,4.0,250000.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1

In [76]:
feather.write_dataframe(df, './vars/df.feather')

## First Draft
This was my process for the first draft of my house prediction, we can revisit this later once we found some insights about the data.